In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (136 kB/s)
Reading package lists... Done


In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-11-20 16:27:54--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.01MB/s    in 0.2s    

2020-11-20 16:27:54 (5.01 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("OperationPolarBear").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
#from pyspark import SparkFiles
from pyspark import SparkFiles
url="https://jim2020.s3.us-east-2.amazonaws.com/pB_2009_2016.csv"
spark.sparkContext.addFile(url)
pB1_2009_2016_df = spark.read.csv(SparkFiles.get("pB_2009_2016.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
pB_2009_2016_df = pB1_2009_2016_df.drop('_c9','_c10','_c11')
pB_2009_2016_df.show()

+--------+---------+--------+------------+-----------+-------+------------+----------+-------+
| bear_id|  GMTdate| GMTtime|        long|        lat|raw_act|standard_act|active_den|habitat|
+--------+---------+--------+------------+-----------+-------+------------+----------+-------+
|pb_20132|4/29/2009| 8:30:00|-148.0326888|70.52573552|  131.0|       131.0|         0|   land|
|pb_20132|4/29/2009| 9:00:00|-148.0327367|70.52573183|  194.0|       194.0|         0|   land|
|pb_20132|4/29/2009| 9:30:00|-148.0327791|70.52577487|    9.0|         9.0|         0|   land|
|pb_20132|4/29/2009|10:00:00|-148.0328216|70.52581791|  205.0|       205.0|         0|   land|
|pb_20132|4/29/2009|10:30:00|-148.0327916| 70.5258169|  542.0|       542.0|         0|   land|
|pb_20132|4/29/2009|11:00:00|-148.0327617| 70.5258159|  494.0|       494.0|         0|   land|
|pb_20132|4/29/2009|11:30:00|-148.0327961|70.52581656|   78.0|        78.0|         0|   land|
|pb_20132|4/29/2009|12:00:00|-148.0328306|70.52581

In [ ]:
pB_den_df = pB1_2009_2016_df.drop("GMTtime", "long", "lat", "raw_act", "standard_act", '_c9','_c10','_c11')
pB_den_df.show()

+--------+---------+----------+-------+
| bear_id|  GMTdate|active_den|habitat|
+--------+---------+----------+-------+
|pb_20132|4/29/2009|         0|   land|
|pb_20132|4/29/2009|         0|   land|
|pb_20132|4/29/2009|         0|   land|
|pb_20132|4/29/2009|         0|   land|
|pb_20132|4/29/2009|         0|   land|
|pb_20132|4/29/2009|         0|   land|
|pb_20132|4/29/2009|         0|   land|
|pb_20132|4/29/2009|         0|   land|
|pb_20132|4/29/2009|         0|   land|
|pb_20132|4/29/2009|         0|   land|
|pb_20132|4/29/2009|         0|   land|
|pb_20132|4/29/2009|         0|   land|
|pb_20132|4/29/2009|         0|   land|
|pb_20132|4/29/2009|         0|   land|
|pb_20132|4/29/2009|         0|   land|
|pb_20132|4/29/2009|         0|   land|
|pb_20132|4/29/2009|         0|   land|
|pb_20132|4/29/2009|         0|   land|
|pb_20132|4/29/2009|         0|   land|
|pb_20132|4/29/2009|         0|   land|
+--------+---------+----------+-------+
only showing top 20 rows



In [ ]:
pB_den_df.printSchema()

root
 |-- bear_id: string (nullable = true)
 |-- GMTdate: string (nullable = true)
 |-- active_den: integer (nullable = true)
 |-- habitat: string (nullable = true)



In [ ]:
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType

#Set user defined function
func =  udf (lambda x: datetime.strptime(x, '%m/%d/%Y'), DateType())

pB_den_df2 = pB_den_df.withColumn('Date', func(col('GMTdate')))
pB_den_df2.show()

+--------+---------+----------+-------+----------+
| bear_id|  GMTdate|active_den|habitat|      Date|
+--------+---------+----------+-------+----------+
|pb_20132|4/29/2009|         0|   land|2009-04-29|
|pb_20132|4/29/2009|         0|   land|2009-04-29|
|pb_20132|4/29/2009|         0|   land|2009-04-29|
|pb_20132|4/29/2009|         0|   land|2009-04-29|
|pb_20132|4/29/2009|         0|   land|2009-04-29|
|pb_20132|4/29/2009|         0|   land|2009-04-29|
|pb_20132|4/29/2009|         0|   land|2009-04-29|
|pb_20132|4/29/2009|         0|   land|2009-04-29|
|pb_20132|4/29/2009|         0|   land|2009-04-29|
|pb_20132|4/29/2009|         0|   land|2009-04-29|
|pb_20132|4/29/2009|         0|   land|2009-04-29|
|pb_20132|4/29/2009|         0|   land|2009-04-29|
|pb_20132|4/29/2009|         0|   land|2009-04-29|
|pb_20132|4/29/2009|         0|   land|2009-04-29|
|pb_20132|4/29/2009|         0|   land|2009-04-29|
|pb_20132|4/29/2009|         0|   land|2009-04-29|
|pb_20132|4/29/2009|         0|

In [ ]:
from pyspark.sql.functions import year, month
pB_den_df3 = pB_den_df2.select("bear_id",year("Date").alias("year"), month("Date").alias("month"), "active_den", "habitat")
pB_den_df3.show()

+--------+----+-----+----------+-------+
| bear_id|year|month|active_den|habitat|
+--------+----+-----+----------+-------+
|pb_20132|2009|    4|         0|   land|
|pb_20132|2009|    4|         0|   land|
|pb_20132|2009|    4|         0|   land|
|pb_20132|2009|    4|         0|   land|
|pb_20132|2009|    4|         0|   land|
|pb_20132|2009|    4|         0|   land|
|pb_20132|2009|    4|         0|   land|
|pb_20132|2009|    4|         0|   land|
|pb_20132|2009|    4|         0|   land|
|pb_20132|2009|    4|         0|   land|
|pb_20132|2009|    4|         0|   land|
|pb_20132|2009|    4|         0|   land|
|pb_20132|2009|    4|         0|   land|
|pb_20132|2009|    4|         0|   land|
|pb_20132|2009|    4|         0|   land|
|pb_20132|2009|    4|         0|   land|
|pb_20132|2009|    4|         0|   land|
|pb_20132|2009|    4|         0|   land|
|pb_20132|2009|    4|         0|   land|
|pb_20132|2009|    4|         0|   land|
+--------+----+-----+----------+-------+
only showing top

In [ ]:
#Create an index for the habitat for aggregation
## Ended up going a different route (left syntax for learnign purposes)

# from pyspark.ml.feature import StringIndexer
# indexer = StringIndexer(inputCol = "habitat", outputCol = "habitatIndex")
# indexed = indexer.fit(pB_den_df3).transform(pB_den_df3)
# indexed.show()

In [ ]:
#Group by month and year
from pyspark.sql import functions as F

pB_den_df3 = pB_den_df3.withColumn("land", F.when(col("habitat") == 'land', 1).otherwise(0))
pB_den_df3 = pB_den_df3.withColumn("ice", F.when(col("habitat") == 'ice', 1).otherwise(0))
pB_den_df3.show()


+--------+----+-----+----------+-------+----+---+
| bear_id|year|month|active_den|habitat|land|ice|
+--------+----+-----+----------+-------+----+---+
|pb_20132|2009|    4|         0|   land|   1|  0|
|pb_20132|2009|    4|         0|   land|   1|  0|
|pb_20132|2009|    4|         0|   land|   1|  0|
|pb_20132|2009|    4|         0|   land|   1|  0|
|pb_20132|2009|    4|         0|   land|   1|  0|
|pb_20132|2009|    4|         0|   land|   1|  0|
|pb_20132|2009|    4|         0|   land|   1|  0|
|pb_20132|2009|    4|         0|   land|   1|  0|
|pb_20132|2009|    4|         0|   land|   1|  0|
|pb_20132|2009|    4|         0|   land|   1|  0|
|pb_20132|2009|    4|         0|   land|   1|  0|
|pb_20132|2009|    4|         0|   land|   1|  0|
|pb_20132|2009|    4|         0|   land|   1|  0|
|pb_20132|2009|    4|         0|   land|   1|  0|
|pb_20132|2009|    4|         0|   land|   1|  0|
|pb_20132|2009|    4|         0|   land|   1|  0|
|pb_20132|2009|    4|         0|   land|   1|  0|


In [ ]:
# Write DataFrame to pB_dens_1910_2018 table in RDS
pB_den_agg = pB_den_df3.select(["year", "month", "active_den", "land", "ice"])\
  .groupby("year", "month")\
  .agg({"active_den": "sum", "land": "sum", "ice": "sum"})\
  .orderBy("year", "month")
pB_den_agg.show()

+----+-----+---------+--------+---------------+
|year|month|sum(land)|sum(ice)|sum(active_den)|
+----+-----+---------+--------+---------------+
|2009|    4|       59|     775|              0|
|2009|    5|      490|   16192|              0|
|2009|    6|        0|   19622|              0|
|2009|    7|      640|   16163|              0|
|2009|    8|     4401|   11665|              0|
|2009|    9|     1499|   11596|              0|
|2009|   10|     1101|   10118|             83|
|2009|   11|       99|    5920|              0|
|2009|   12|      141|    5459|              0|
|2010|    1|        5|    4707|              0|
|2010|    2|       16|    3999|              0|
|2010|    3|      120|    3941|              0|
|2010|    4|       94|    1559|              0|
|2010|    5|        0|     840|              0|
|2011|    3|        0|      30|              0|
|2011|    4|       17|    1387|              0|
|2011|    5|       49|    1993|              0|
|2011|    6|       53|    1446|         

In [ ]:
pB_den_agg = pB_den_agg.withColumnRenamed("sum(land)", "num_land_dens")\
.withColumnRenamed("sum(ice)", "num_ice_dens")\
.withColumnRenamed("sum(active_den)", "num_active_dens")

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://polarbear.c6c40hihvfwz.us-east-2.rds.amazonaws.com:5432/polarbear_db"
config = {"user":"root", 
          "password": "Welcome1", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table
pB_den_agg.write.jdbc(url=jdbc_url, table='public.pb_den_agg', mode=mode, properties=config)